# Image classification using TensorFlow estimators and TensorFlow Hub for transfer learning

*This notebook is available [as a codelab](https://colab.research.google.com/github/damienpontifex/BlogCodeSamples/blob/master/TransferLearning/TfHubDogsVsCats.ipynb)*

TensorFlow Hub was [announced](https://www.youtube.com/watch?v=rirzJ-e68cw) at TensorFlow Dev Summit 2018 and promises to reduce the effort required to use existing machine learning models and weights in your own custom model. From the [overview page](https://www.tensorflow.org/hub/)

> TensorFlow Hub is a library to foster the publication, discovery, and consumption of reusable parts of machine learning models. A module is a self-contained piece of a TensorFlow graph, along with its weights and assets, that can be reused across different tasks in a process known as transfer learning.

Dogs vs Cats is a great classification problem to learn about transfer learning and is the [first lesson of the fast.ai course](http://course.fast.ai/lessons/lesson1.html) and was hosted on Kaggle. The teaching is if you can achieve transfer learning for this two-class problem of cats and dogs, you can do it for any *n*-class problem for your own solution.

### Prerequisites

You will need to install TensorFlow Hub from pip and have at least TensorFlow version 1.7. I'm also assuming you have some understanding (although not required but will be in this code) of TensorFlow and machine learning, mainly:
* Data preparation
* tf.data input
* TensorFlow estimators

To ensure you've got the libraries installed run:

In [2]:
%%bash 
# pip3 install -qU tensorflow>=1.7.0 # Don't run this on Google's colab as you probably want to use the version they have pre-installed
python -m pip install -qU tensorflow-hub

kaggle 1.4.7.1 requires tqdm, which is not installed.
kaggle 1.4.7.1 has requirement urllib3<1.23.0,>=1.15, but you'll have urllib3 1.23 which is incompatible.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Just some usual imports and matplotlib setup for notebooks. And then just print out some information about our TensorFlow environment

In [3]:
import os
import tensorflow as tf
import tensorflow_hub as hub

l = tf.keras.layers

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
tf.logging.info('TF Version {}'.format(tf.__version__))
tf.logging.info('GPU Available {}'.format(tf.test.is_gpu_available()))
if 'TF_CONFIG' in os.environ:
    tf.logging.info('TF_CONFIG: {}'.format(os.environ["TF_CONFIG"]))

INFO:tensorflow:TF Version 1.12.0
INFO:tensorflow:GPU Available False


The Dogs vs Cats Kaggle competition is a two-class image classification problem. Transfer learning can be used to reduce the amount of computation and to reuse previously computed features of interest.

For this, we'll look into the new TensorFlow Hub modules to train on our own dataset. First lets just grab, and have a look, at the data we've got.

## Data

We will get the data from the fastai zip of the dogs and cats images. The file pattern on disk is to have each category of images labelled by placing all associated images into one folder. As such, we will have a folder named 'dogs' of all dog images and 'cats' of all cat images to train on.

So that this notebook can be run anywhere, here we are just downloading the archive and placing it in a temporary local data directory

In [ ]:
def get_data(local_data_root: str, is_chief: bool=True):
    data_dir = os.path.join(local_data_root, 'datasets/dogscats')
    
    if is_chief:
        if not tf.gfile.IsDirectory(data_dir):
            # Download the data zip to our data directory and extract
            fallback_url = 'http://files.fast.ai/data/dogscats.zip'
            tf.keras.utils.get_file(
                os.path.join(local_data_root, os.path.basename(fallback_url)), 
                fallback_url, 
                cache_dir=local_data_root,
                extract=True)
        
    return data_dir

In [ ]:
data_dir = get_data('/tmp', True)

Looking at what is in the data directory

In [ ]:
tf.gfile.ListDirectory(data_dir)

Look in the valid directory to see the image classes. Here we see we have a folder of dogs and another of cats.

In [ ]:
tf.gfile.ListDirectory(os.path.join(data_dir, 'valid'))

In [ ]:
cats = tf.gfile.ListDirectory(os.path.join(data_dir, 'valid', 'cats'))[:5]
cats

In [ ]:
with tf.gfile.GFile(os.path.join(data_dir, 'valid', 'cats', cats[0]), 'rb') as f:
    img = plt.imread(f)
    plt.imshow(img)

In [ ]:
img.shape

## tf.data to access our dataset

Our dataset input function is responsible for providing features and labels to the network. We will use a glob pattern via the `file_pattern` parameter to indicate the files on disk and return a dataset to use.

In [4]:
def _img_string_to_tensor(image_string, image_size=(299, 299)):
    """Decodes jpeg image bytes and resizes into float32 tensor
    
    Args:
      image_string: A Tensor of type string that has the image bytes
    
    Returns:
      float32 tensor of the image
    """
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    # Convert from full range of uint8 to range [0,1] of float32.
    image_decoded_as_float = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
    # Resize to expected
    image_resized = tf.image.resize_images(image_decoded_as_float, size=image_size)
    
    return image_resized

In [ ]:
def make_dataset(file_pattern, image_size=(299, 299), shuffle=False, batch_size=64, num_epochs=None, buffer_size=4096):
    """Makes a dataset reading the input images given the file pattern
    
    Args:
      file_pattern: File pattern to match input files with
      image_size: size to resize images to
      shuffle: whether to shuffle the dataset
      batch_size: the batch size of the dataset
      num_epochs: number of times to repeat iteration of the dataset
      buffer_size: size of buffer for prefetch and shuffle operations
    
    Returns:
      A tf.data.Dataset with dictionary of key to Tensor for features and label Tensor of type string
    """
    
    def _path_to_img(path):
        """From the given path returns a feature dictionary and label pair
        
        Args:
          path: A Tensor of type string of the file path to read from
          
        Returns:
          Tuple of dict and tensor. 
          Dictionary is key to tensor mapping of features
          Label is a Tensor of type string that is the label for these features
        """
        # Get the parent folder of this file to get it's class name
        label = tf.string_split([path], delimiter='/').values[-2]
        
        # Read in the image from disk
        image_string = tf.io.read_file(path)
        image_resized = _img_string_to_tensor(image_string, image_size)
        
        return { 'image': image_resized }, label
    
    opt = tf.data.Options()
    opt.experimental_autotune = True
    opt.experimental_map_and_batch_fusion = True
    opt.experimental_shuffle_and_repeat_fusion = True
    
    dataset = tf.data.Dataset.list_files(file_pattern)

    if shuffle:
        dataset = dataset.shuffle(buffer_size)

    dataset = dataset.repeat(num_epochs)
    dataset = dataset.map(_path_to_img)
    dataset = dataset.batch(batch_size).prefetch(buffer_size)

    dataset = dataset.with_options(opt)

    return dataset

## Our first model

We're setting up an esimator `model_fn` so we can include the module as the base and then provide our own dense layer and activations to be trained for our use case. We're passing in the module spec (a string) via the `params` dictionary so we could swap out another image classification model easily.

The highlight here for TensorFlow Hub and transfer learning are these lines:

```python
module = hub.Module(params['module_spec'], trainable=is_training, name=params['module_name'])
bottleneck_tensor = module(features['image'])
```

We load up the module and specify whether we would like to fine tune it or not. Then pass in our image tensor to the module and get the output tensor as `bottleneck_tensor` to be used in further layers. If you see a diagram of the structure of some of these models, the simplification of this down to two lines is amazing!

In [ ]:
def model_fn(features, labels, mode, params):
    """tf.estimator model function implementation for retraining an image classifier from a 
    tf hub module
    
    Args:
      features: dictionary of key to Tensor
      labels: Tensor of type string
      mode: estimator mode
      params: dictionary of parameters
      
    Returns:
      tf.estimator.EstimatorSpec instance
    """
    is_training = mode == tf.estimator.ModeKeys.TRAIN
    module_trainable = is_training and params.get('train_module', False)

    module = hub.Module(params['module_spec'], trainable=module_trainable, name=params['module_name'])
    bottleneck_tensor = module(features['image'])
    
    NUM_CLASSES = len(params['label_vocab'])
    logit_units = 1 if NUM_CLASSES == 2 else NUM_CLASSES
    logits = l.Dense(logit_units)(bottleneck_tensor)

    if NUM_CLASSES == 2:
        head = tf.contrib.estimator.binary_classification_head(label_vocabulary=params['label_vocab'])
    else:
        head = tf.contrib.estimator.multi_class_head(n_classes=NUM_CLASSES, label_vocabulary=params['label_vocab'])

    optimizer = tf.train.AdamOptimizer(learning_rate=params.get('learning_rate', 1e-3))
    
    return head.create_estimator_spec(
        features, mode, logits, labels, optimizer=optimizer
    )

Finally, we setup our estimator as per usual:

* Define some hyperparameters
* Construct the estimator
* `train_and_evaluate`

And with that, we get distributed training and great device placement for data processing on the CPU and training on the GPU.

In [ ]:
def train():
    """Run training operation
    """
    
    run_config = tf.estimator.RunConfig()
    
    data_directory = get_data('/tmp', run_config.is_chief)
    model_directory = '/tmp/dogscats/run2'

    params = {
        'module_spec': 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/1',
        'module_name': 'resnet_v2_50',
        'learning_rate': 1e-3,
        'train_module': False,  # Whether we want to finetune the module
        'label_vocab': tf.gfile.ListDirectory(os.path.join(data_directory, 'valid'))
    }

    classifier = tf.estimator.Estimator(
        model_fn=model_fn,
        model_dir=model_directory,
        config=run_config,
        params=params
    )

    input_img_size = hub.get_expected_image_size(hub.Module(params['module_spec']))

    train_files = os.path.join(data_directory, 'train', '**/*.jpg')
    train_input_fn = lambda: make_dataset(train_files, image_size=input_img_size, batch_size=8, shuffle=True)
    train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps=30)

    eval_files = os.path.join(data_directory, 'valid', '**/*.jpg')
    eval_input_fn = lambda: make_dataset(eval_files, image_size=input_img_size, batch_size=1)
    eval_spec = tf.estimator.EvalSpec(eval_input_fn)

    tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

In [ ]:
train()

## Conclusion

TensorFlow Hub will significantely increase the approachability for people to use complex models from others for their own specific task. The ability to fine tune while also using as is continues to democritize everyone's use of ML tools.